In [36]:
%run ./Binance_keys.ipynb
from binance import Client
import pandas as pd
client = Client(api_key, secret_key)

In [35]:
posframe = pd.read_csv('positioncheck')

In [37]:
def gethourlydata(symbol):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1h', '60 hours ago UTC'))
    frame = frame[[0,4]]
    frame.columns = ['time', 'close']
    return frame

In [38]:
ethHourly = gethourlydata('ETHUSDT')

In [39]:
def applyFSSMA(df):
    df['fastSMA'] = df.close.rolling(8).mean()
    df['slowSMA'] = df.close.rolling(21).mean()

In [48]:
applyFSSMA(ethHourly)

In [42]:
def changepos(coin, order, buy=True):
    if buy:
        posframe.loc[posframe.currency == coin, 'position'] = 1
        posframe.loc[posframe.currency == coin, 'quantity'] = float(order['executedqty'])
    else:
        posframe.loc[posframe.currency == coin, 'position'] = 0
        posframe.loc[posframe.currency == coin, 'quantity'] = 0

In [51]:
#investment amount in USD
def trader(usd=100):
    for coin in posframe[posframe.position == 1].currency:
        print('test')
        coindata = gethourlydata(coin)
        applyFSSMA(coindata)
        latestdata = coindata.iloc[-1]
        if latestdata.slowSMA > latestdata.fastSMA:
            order = client.create_order(symbol=coin, side='SELL', type='MARKET',
            qty = posframe[posframe.currency == coin].quantity.values[0])
            changepos(coin,order,buy=False)
    for coin in posframe[posframe.position == 0].currency:
        coindata = gethourlydata(coin)
        applyFSSMA(coindata)
        latestdata = coindata.iloc[-1]
        if latestdata.fastSMA > latestdata.slowSMA:
            order = client.create_order(symbol=coin, side='BUY', type='MARKET', quoteOrderqty=usd)
            print(order)
            changepos(coin,order,buy=True)
        else:
            print(f'did not buy {coin}')

In [52]:
trader(0)

did not buy BTCUSDT
did not buy ETHUSDT
did not buy BNBUSDT
did not buy ADAUSDT
did not buy DOGEUSDT
